In [13]:
%matplotlib notebook
import os
import os.path as path
import numpy as np
from scipy.optimize import curve_fit
from faker import Factory
from scipy import interpolate
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math 
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
import h5py

from pyccapt.calibration.calibration_tools import tools, data_tools, variables, tof_calibration, data_loadcrop
from pyccapt.calibration.calibration_tools import widgets as wd


In [2]:
number_sample = 300
save_fig=False
plot_fig=True 

In [3]:
# flightPathLength = 110 # mm 
# # The initial value for t_0
# t0 = 51.74 # ns

# dataset name
# AL_data_b
# OLO_AL_6_data
# OLO_W_6_data
# OLO_Ni_8_data
# X6Cr17_2V30Min_5_data
# data_1029_Jun-27-2022_15-53_Pd
# data_115_Jul-27-2022_17-44_Powersweep3

In [4]:
tdc, pulse_mode, dataset, flightPathLength_d, t0_d = wd.dataset_tdc_selection()
display(tdc, pulse_mode, dataset, flightPathLength_d, t0_d)

Dropdown(description='TDC model:', options=('surface_concept', 'roentdec'), value='surface_concept')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

Text(value='OLO_AL_6_data', description='Dataset:', placeholder='Paste ticket description here!')

Text(value='110', description='Flight path length:', placeholder='Flight path length')

Text(value='51.74', description='t0:', placeholder='T_0 of the instrument')

$$\textbf{You can specify which dataset to use in below block}$$

In [6]:
tdc_model = tdc.value
pulse_mode_ini = pulse_mode.value

dataset_name = dataset.value

flightPathLength = float(flightPathLength_d.value) # mm 
# The initial value for t_0
t0 = float(t0_d.value) # ns

p = path.abspath(path.join("", "../../../.."))

variables.init()

path_main = os.path.join(p, 'tests//data')
filename_main = path_main + '//' + dataset_name + '.h5'

# variables.path = os.path.join(p, 'tests//results//mc_vol_bowl_calibratin')
variables.path = os.path.join(p, 'tests//results//tof_calibration')
variables.result_path = os.path.join(p, 'tests/results/ion_type_selection/' + dataset_name)
if not os.path.isdir(variables.result_path):
        os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
        
filename = variables.path + '//' + dataset_name + '//' + dataset_name + '.h5'



head, tail = os.path.split(filename)
figname = os.path.splitext(tail)[0]

data = data_tools.read_hdf5_through_pandas(filename)


In [7]:
data

,high_voltage (V),pulse (V),start_counter,t (ns),x (mm),y (mm),pulse_pi,ion_pp,mc (Da)
0,5239.389648,5239.389648,20476.0,626.265702,5.762449,4.329796,0.0,1.0,27.165212
1,5239.389648,5239.389648,21274.0,648.087858,-11.206531,26.902041,798.0,1.0,27.169202
2,5239.389648,5239.389648,22584.0,639.583938,24.259592,15.504490,1310.0,1.0,27.094883
3,5239.389648,5239.389648,22746.0,627.959628,18.083265,-7.545306,162.0,1.0,26.998025
4,5239.389648,5239.389648,23789.0,623.357910,-1.942041,-2.706122,1043.0,1.0,26.986939
...,...,...,...,...,...,...,...,...,...
1926403,5710.397949,5710.397949,19724.0,609.401880,-15.886531,-9.742041,77.0,1.0,27.134127
1926404,5710.397949,5710.397949,19742.0,601.796358,11.397551,-5.475918,18.0,1.0,26.995510
1926405,5710.397949,5710.397949,19969.0,455.220324,31.008980,-1.719184,227.0,1.0,13.597599
1926406,5710.397949,5710.397949,20649.0,611.857044,2.483265,-27.284082,680.0,1.0,27.061553


In [8]:
mc = data['mc (Da)'].to_numpy()

In [29]:
peaks_sides_p = np.zeros(0)
def mc_his(bin_size, prominence, distance, percent, plot, figname, mc, mc_ideal):

    bin_size = float(bin_size)
    prominence = int(prominence)
    distance = int(distance)
    percent = int(percent)
    if mc_ideal == True:
        mc_ideal = np.array(variables.listMaterial)
        print('List of mc are:', mc_ideal)
    else:
        mc_ideal = np.zeros(0)
    max_hist, left_right_peaks, peaks_sides, max_paek_edges, index_max = tools.massSpecPlot(mc[mc < 100], bin_size, mc_ideal, distance=distance, percent=percent, prominence=prominence, plot=plot, label='tof', fig_name=figname)
    mrp = (max_hist / (left_right_peaks[1] - left_right_peaks[0]))
    print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
    for i in range(len(peaks_sides)):
        print('Peaks ', i, 'is: {:.2f}'.format(peaks_sides[i,0]), 'peak window sides are: {:.2f} - {:.2f}'.format(peaks_sides[i,2], peaks_sides[i,3]))
    global peaks_sides_p
    peaks_sides_p = peaks_sides

In [30]:
peaks_sides = interact_manual(mc_his, bin_size=widgets.Textarea(value="0.1")
               ,prominence=widgets.Textarea(value="60"), distance=widgets.Textarea(value="100")
               ,percent=widgets.Textarea(value="20"), plot=fixed(True), mc=fixed(mc),  figname=fixed(figname)
                , mc_ideal=[('False',False),('True',True)]);



interactive(children=(Textarea(value='0.1', description='bin_size'), Textarea(value='60', description='promine…

In [11]:
isotopeTableFile = '../../../files/isotopeTable.h5'
dataframe = data_tools.read_hdf5_through_pandas(isotopeTableFile)
elementsList = dataframe['element']
elementIsotopeList = dataframe['isotope']
elementMassList =  dataframe['weight']
abundanceList = dataframe['abundance']

elements = list(zip(elementsList, elementIsotopeList, elementMassList, abundanceList))
dropdownList = []
for element in elements:
    tupleElement = ("{} ({}) ({:.2f})".format(element[0],element[1],element[3]), "{}{}".format(element[0],element[1]))
    dropdownList.append(tupleElement)

chargeList = [(1,1,),(2,2,),(3,3,),(4,4,)]
dropdown = wd.dropdownWidget(dropdownList, "Elements")
dropdown.observe(wd.on_change_ions_selection)


chargeDropdown = wd.dropdownWidget(chargeList, "Charge")
chargeDropdown.observe(wd.on_change_charge_ions_selection)

wd.compute_element_isotope_values_according_to_selected_charge(mode='ions_selection')

buttonAdd = wd.buttonWidget("ADD")
buttonDelete = wd.buttonWidget("DELETE")
buttonReset = wd.buttonWidget("RESET")

display(dropdown)
display(chargeDropdown)
display(buttonAdd)
display(buttonDelete)
display(buttonReset)
listMaterial = buttonAdd.on_click(wd.onClickAdd)
buttonDelete.on_click(wd.onClickDelete)
buttonReset.on_click(wd.onClickReset)

Dropdown(description='Elements', options=(('H (1) (99.98)', 'H1'), ('H (2) (0.01)', 'H2'), ('He (3) (0.00)', '…

Dropdown(description='Charge', options=((1, 1), (2, 2), (3, 3), (4, 4)), value=1)

Button(description='ADD', icon='check', style=ButtonStyle(), tooltip='ADD')

Button(description='DELETE', icon='check', style=ButtonStyle(), tooltip='DELETE')

Button(description='RESET', icon='check', style=ButtonStyle(), tooltip='RESET')

In [33]:
fake = Factory.create()
color = np.zeros(len(mc), dtype='object' )
element = np.zeros(len(mc), dtype='object')

for index, elemen in enumerate(variables.listMaterial):
    mask = np.logical_and((peaks_sides_p[index,2] < mc), (mc < peaks_sides_p[index,3])).squeeze()
    print(mask[mask==True].shape)

    index_true = np.where(mask==True)
    index_get_to_false = np.random.choice(index_true[0], size=int(index_true[0].shape[0] - index_true[0].shape[0]), replace=False)
    mask[index_get_to_false] = False
    print('ploted ions', mask[mask==True].shape)
    
    color[mask] = fake.hex_color()
    element[mask] = elemen



(949,)
ploted ions (949,)
(144725,)
ploted ions (144725,)
(1551591,)
ploted ions (1551591,)


In [36]:
data['element'] = element
data['color'] = color

In [37]:
data

,high_voltage (V),pulse (V),start_counter,t (ns),x (mm),y (mm),pulse_pi,ion_pp,mc (Da),element,color
0,5239.389648,5239.389648,20476.0,626.265702,5.762449,4.329796,0.0,1.0,27.165212,Al27(1+),#8fcea3
1,5239.389648,5239.389648,21274.0,648.087858,-11.206531,26.902041,798.0,1.0,27.169202,Al27(1+),#8fcea3
2,5239.389648,5239.389648,22584.0,639.583938,24.259592,15.504490,1310.0,1.0,27.094883,Al27(1+),#8fcea3
3,5239.389648,5239.389648,22746.0,627.959628,18.083265,-7.545306,162.0,1.0,26.998025,Al27(1+),#8fcea3
4,5239.389648,5239.389648,23789.0,623.357910,-1.942041,-2.706122,1043.0,1.0,26.986939,Al27(1+),#8fcea3
...,...,...,...,...,...,...,...,...,...,...,...
1926403,5710.397949,5710.397949,19724.0,609.401880,-15.886531,-9.742041,77.0,1.0,27.134127,Al27(1+),#8fcea3
1926404,5710.397949,5710.397949,19742.0,601.796358,11.397551,-5.475918,18.0,1.0,26.995510,Al27(1+),#8fcea3
1926405,5710.397949,5710.397949,19969.0,455.220324,31.008980,-1.719184,227.0,1.0,13.597599,Al27(2+),#45f830
1926406,5710.397949,5710.397949,20649.0,611.857044,2.483265,-27.284082,680.0,1.0,27.061553,Al27(1+),#8fcea3


In [38]:
# save the new data
name_save_file = variables.result_path + '//' + dataset_name + '.h5'
data_tools.store_df_to_hdf(name_save_file, data, 'df')

d:\pyccapt\pyccapt\calibration\calibration_tools\data_tools.py:91: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['element', 'color'], dtype='object')]

  dataframe.to_hdf(filename, key, mode='w')


In [41]:
data.dtypes

high_voltage (V)    float64
pulse (V)           float64
start_counter       float64
t (ns)              float64
x (mm)              float64
y (mm)              float64
pulse_pi            float64
ion_pp              float64
mc (Da)             float64
element              object
color                object
dtype: object